In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from import_casa import casa

In [12]:
import pandas as pd
import pickle
from gensim.models.keyedvectors import KeyedVectors
path = casa.get_data_path() / "cadet/op20/ft-2020.kv"
ft = KeyedVectors.load(str(path))
ft

[INFO] 2021-01-26 18:50:50,610 gensim.utils: loading Word2VecKeyedVectors object from E:\LangOn\casa\data\cadet\op20\ft-2020.kv
[INFO] 2021-01-26 18:50:50,896 gensim.utils: setting ignored attribute vectors_norm to None
[INFO] 2021-01-26 18:50:50,897 gensim.utils: setting ignored attribute vectors_vocab_norm to None
[INFO] 2021-01-26 18:50:50,898 gensim.utils: setting ignored attribute vectors_ngrams_norm to None
[INFO] 2021-01-26 18:50:50,899 gensim.utils: setting ignored attribute buckets_word to None
[INFO] 2021-01-26 18:50:50,899 gensim.utils: loaded E:\LangOn\casa\data\cadet\op20\ft-2020.kv


In [13]:
ft.vectors_ngrams.shape

(30000, 200)

In [38]:
"esim" in ft.vocab

True

In [37]:
ft.most_similar("4g", topn=20)

[('的4g', 0.7653027772903442),
 ('4g的', 0.7631452083587646),
 ('是4g', 0.7176839113235474),
 ('在4g', 0.6869112253189087),
 ('4g網路', 0.6786275506019592),
 ('4g網速', 0.6701086163520813),
 ('4g吃到飽', 0.6615954637527466),
 ('/4g', 0.6571164727210999),
 ('4g+', 0.6448955535888672),
 ('4g速度', 0.6426206231117249),
 ('4g手機', 0.6417348384857178),
 ('用4g', 0.6400904655456543),
 ('4g方案', 0.6369199156761169),
 ('中華4g', 0.6171366572380066),
 ('3g', 0.6161083579063416),
 ('4g資費', 0.6143298745155334),
 ('4g都', 0.6138687133789062),
 ('跟4g', 0.6111692786216736),
 ('4g訊號', 0.6053209900856018),
 ('止', 0.59618079662323)]

In [5]:
seeds = pd.read_excel(casa.get_data_path()/"seeded_spm_edited.xlsx")

In [24]:
from casa.topics import SeedEntry, SeedDistribution
seed_entries = []
for _, row in seeds.iterrows():
    candidates = [str(x) for x in row[2:].dropna()]
    entry = SeedEntry(row.Category, row.Seed, candidates)
    seed_entries.append(entry)    

In [ ]:
## collective nouns is ignored: 三雄, 三大電信, 電信三雄
entities_seeds = [
    SeedEntry("電信業者", "中華電信", "中華電信 中華電 中華 種花"),
    SeedEntry("電信業者", "遠傳電信", "遠傳電信 遠傳 遠傳5G"),
    SeedEntry("電信業者", "台灣大哥大", "台灣大哥大 台灣大 台哥大 台哥"),
    SeedEntry("電信業者", "台灣之星", "台灣之星"),
    SeedEntry("電信業者", "亞太", "亞太 亞太電信 亞太電 亞太5G"),    
]

In [27]:
seed_distr = SeedDistribution(model.wv, seed_entries)

In [35]:
seed_distr.get_distribution("中華電信好爛").shape

(880, 4)

## Topic Seeds

In [106]:
from casa import Cadet
import numpy as np

In [107]:
cadet = Cadet.load(casa.get_data_path()/"cadet/op20")
cadet

[INFO] 2021-01-27 00:20:19,695 gensim.utils: loading Word2VecKeyedVectors object from E:\LangOn\casa\data\cadet\op20\ft-2020.kv
[INFO] 2021-01-27 00:20:19,996 gensim.utils: setting ignored attribute vectors_norm to None
[INFO] 2021-01-27 00:20:19,997 gensim.utils: setting ignored attribute vectors_vocab_norm to None
[INFO] 2021-01-27 00:20:19,998 gensim.utils: setting ignored attribute vectors_ngrams_norm to None
[INFO] 2021-01-27 00:20:19,998 gensim.utils: setting ignored attribute buckets_word to None
[INFO] 2021-01-27 00:20:19,999 gensim.utils: loaded E:\LangOn\casa\data\cadet\op20\ft-2020.kv
[INFO] 2021-01-27 00:20:20,000 gensim.models.keyedvectors: precomputing L2-norms of word weight vectors
[INFO] 2021-01-27 00:20:20,009 gensim.models.keyedvectors: precomputing L2-norms of ngram weight vectors


<Cadet: FastText(9851, 200), Seeds(100)>

In [108]:
vec = cadet.build_seedsims_matrix("中華電信的4g很便宜").max(axis=0)

tokens:  ['中華電信', '的4g', '很便宜']


In [112]:
cadet.detect("果然打電話客訴有用 昨天晚上網路 測速比之前還快 晚上還能穩定15MB = =")

tokens:  ['果然', '打電話', '客訴', '有用', '▁昨天', '晚上', '網路', '▁', '測速', '比', '之前', '還', '快', '▁晚上', '還能', '穩定', '15', 'mb', '▁=', '▁=']


{'entity': ['中華電信', '台灣大哥大', '遠傳電信', '台灣之星', '亞太電信', '其他電信'],
 'entity_probs': array([0.1672673 , 0.17283326, 0.16449066, 0.16464284, 0.16505646,
        0.16570949]),
 'service': ['資費方案', '通訊品質', '加值服務'],
 'service_probs': array([9.16078501e-05, 9.99814418e-01, 9.39741130e-05]),
 'seeds': ['速度', '測速', '測試', '上傳', '下載'],
 'seed_probs': array([0.1249125, 0.1249125, 0.1249125, 0.1249125, 0.1249125])}

In [93]:
np.argsort([1,3,2])

array([0, 2, 1], dtype=int64)

In [102]:
cadet.kv.most_similar("遠傳電信", topn=20)

[('、遠傳電信', 0.8342025279998779),
 ('中華電信、遠傳電信', 0.8040480613708496),
 ('。遠傳電信', 0.7685631513595581),
 ('亞太電信', 0.7522163391113281),
 ('中華電信、台灣大哥大', 0.731809139251709),
 ('台灣大哥大', 0.7297022938728333),
 ('、台灣大哥大', 0.697385311126709),
 ('、亞太電信', 0.6964842081069946),
 ('與台灣大哥大', 0.6867240071296692),
 ('。台灣大哥大', 0.6790187358856201),
 ('中華電信', 0.6725145578384399),
 ('揀', 0.6520723104476929),
 ('、台灣之星', 0.6196701526641846),
 ('台灣大哥大的', 0.5919278860092163),
 ('瞥', 0.5867152810096741),
 ('。中華電信', 0.5778483748435974),
 ('。亞太電信', 0.577357292175293),
 ('氯錠', 0.5697625875473022),
 ('、遠傳', 0.5667248964309692),
 ('、台灣大', 0.5474036931991577)]